In [ ]:
using Distributed
using Revise
parallel = false
push!(LOAD_PATH, "$(homedir())/Development/")
if parallel
    addprocs(4)
    @everywhere push!(LOAD_PATH, "$(homedir())/Development/")
    @everywhere using JSort
else
    using JSort
end

In [ ]:
sortfile("si28.yaml")

# Plot the quality of the $^{28}Si$ data file

In [ ]:
using Plots

In [ ]:
data = read("quality.dat") |> x -> reinterpret(Int8, x);

In [ ]:
function findsquare(N)
    i = 0
    while i^2 < N
        i += 1
    end
    return i
end

#n = findsquare(length(data))
#n = 100
#sdata = zeros(Int8, (n, n)) .- Int8(2)
#m = 20000
#sdata[1:n^2] = data[m:m+n^2-1];

In [ ]:
n = 150
m = 20000
heatmap(1:n, 1:n, data[m:m+n^2-1])

# Alignment

In [ ]:
using Revise
push!(LOAD_PATH, "$(homedir())/Development/")
using JSort
using Plots
using DSP

In [ ]:
events = loadlabr("sirius");
parameters = Parameters("si28.yaml");

In [ ]:
ge, gt = makelabr(events, parameters);

In [ ]:
mat = copy(ge.matrix);
chns = goodchannels(mat);
G = mat[:, chns];
M = [G[:, i] for i in 1:size(G, 2)];

In [ ]:
function plotmat(matrix)
    G = convert(Array{Float64, 2}, matrix)
    log10transform!(G)
    pl = heatmap(G', c=:viridis, yticks=([1:length(chns);], chns))
end

function log10transform!(G)
    G[G .> 0] .= log10.(G[G .> 0])
    G[G .≤ 0] .= NaN
end

function plotstrips(g, I)
    G = convert(Array{Float64, 2}, g)
    log10transform!(G)
    pl = heatmap(G', c=:viridis, yticks=([1:length(I);], I))#, c=:viridis, yticks=([1,2], [i,j]))
    #pl = heatmap!(Y, c=:viridis)
end

In [ ]:
plotmat(G)

In [ ]:
G = mat[:, chns]
pl = plotmat(G)

for (i, chn) in enumerate(chns)
    numpeaks = 3
    peaks = findpeaks(G[:, i], 100:lastindex(G[:, i]), numpeaks=numpeaks)
    pl = scatter!(peaks, repeat([i,], numpeaks), markershape=:dtriangle, markercolor=:red,
    markerstrokewidth=0)
end
pl

In [ ]:
ref = 1
M = [G[:, i] for i in 1:size(G, 2)]
# Note: Use View to change G inline
numpeaks = 3
for (i, chn) in enumerate(chns)
    ref == i && continue
    shift, gain = linearalign(M[i], M[ref], 100:lastindex(M[ref]), numpeaks=numpeaks)
    @show i, chn, shift, gain
    gainshift!(M[i], shift, gain);
end

In [ ]:
plotmat(hcat(M...))

In [ ]:
residuals = (M[1] .- M[2]).^2 ./(M[1]).^2
scatter(residuals[isfinite.(residuals)], markerstrokewidth=0, markersize=1)

In [ ]:
using Optim

In [ ]:
A = copy(M[ref])
B = copy(M[3])
C = signal.savgol_filter(A, 51, 3)
D = signal.savgol_filter(B, 51, 3)
plot(A, alpha=0.2)
plot!(B, alpha=0.2)
plot!(C)

plot!(D)

In [ ]:
#NOTE: The smoothing create negative bins, which are invalid for creating random numbers in gainshift
roi = [500:2000;]
C = round.(Int, C)
C[C .< 0] .= 0
D = round.(Int, D)
D[D .< 0] .= 0
plot(A[roi], alpha=0.2)
plot!(B[roi], alpha=0.2)
plot!(C[roi])
plot!(D[roi])
X, Y = C[roi], D[roi];

In [ ]:
function χ²(target, ref)
    S = @. (ref-target)^2/ref^2
    sum(S[isfinite.(S)])
end

#const P1 = [10_000 100 1]
lower = [-150.0 0.92 -1e-4]
upper = [150.0 1.08 1e-4]

function errfn(x)
    #@. x *= P1
    #@show x
    #if !all(lower .<= x .<= upper)
    #    return 1e9
    #end
    Z = signal.savgol_filter(gainshift(Y, x...), 51, 3)
    χ²(Z, X)
end


x₀ = [0.0 1.0 0.0] #./ P1
results = optimize(errfn, x₀, method=NelderMead())


In [ ]:
@show Optim.minimizer(results)
Z =signal.savgol_filter(gainshift(Y, 0, 1.0589, 3e-4), 51, 3) 
V = signal.savgol_filter(gainshift(Y, Optim.minimizer(results)...), 51, 3)
plot(X)
plot!(Y)
plt = plot!(Z)
plt = plot!(V)
@show χ²(Y, X)
@show χ²(Z, X)
@show χ²(V, X)
plt

In [ ]:
plot(A)
plot!(gainshift(B, Optim.minimizer(results)...))

In [ ]:
points = localalign(M[2], M[1], [150:1500;])

plt = plotmat(G[:, 1:2])
@show points
for _points in points
    plt = scatter!(_points, [1,2],markershape=:dtriangle, markercolor=:red, markerstrokewidth=0)
end
plt

# Results
The minimization is very finicky to get right. Might be a good idea to skip the
initial minimization and instead do the local feature minimization instead.
Might have to repeat the local minimization several times to get enough points
for a good fit. That the spectrum is not minimized before might lead to areas where
the spectra are so shifted that they can not be aligend properly, so areas with abnormaly large
$\chi^2$ must be excluded.

# Residuals
Measure the residuals in the peak alignment. This is stupid. I need a lot more points

In [ ]:
ref = 1
G = mat[:, chns]
N = [G[:, i] for i in 1:size(G, 2)]

numpeaks = 4
println("here")
reference_peaks = findpeaks(N[ref], 100:lastindex(N[ref]), numpeaks=numpeaks)
println("there")
residuals = Float64[]
l = scatter()
for (i, chn) in enumerate(chns)
    i == ref && continue
    println(i)
    peaks = findpeaks(N[i], 100:lastindex(N[i]), numpeaks=numpeaks)
    @show shift, gain = leastsquares(peaks, reference_peaks)
    @show res = reference_peaks .- (shift .+ gain.*peaks)
    l = scatter!(res)
end
l
#scatter(residuals)

In [ ]:
1+1

# Measure the amount of "features" in a spectrum
- Use this to find the regions of the spectrum to use for local fitting.
- I have found no solution to this

In [ ]:
ref = 1
G = mat[:, chns]
N = [G[:, i] for i in 1:size(G, 2)];
using PyCall
signal = pyimport("scipy.signal")

In [ ]:
X = signal.savgol_filter(N[ref], 51, 3)[500:end]
differentiate(x) = (x[1:end-1].-x[2:end])./(x[2]-x[1])
X′ = signal.savgol_filter(differentiate(X), 51, 3)
X′′ = signal.savgol_filter(differentiate(X′), 51, 3)
smoothdiff(x, window=51) = signal.savgol_filter(differentiate(x), window, 3)
function snratio(spectrum, numregions)
    step = floor.(Int, length(spectrum)/numregions)
    laststep = step+ length(spectrum) - numregions*step
    Q = zeros(Float64, numregions)
    D = spectrum |> smoothdiff |> smoothdiff
    for i in 1:numregions-1
        diff2 = D[((i-1)*step)+1:(i*step)]
        Q[i] = sum(abs.(diff2))
    end
    Q[numregions] = sum(abs.(D[(numregions-1)*step+1:end]))
    Q
end
plt0 = plot([X, X′, X′′], layout=3)
#plt = plot(range(firstindex(X), lastindex(X), length=2), snratio(X, 2), seriestype=:steppre)
#for i in 3:20
#    plt = plot!(range(firstindex(X), lastindex(X), length=i), snratio(X, i), seriestype=:steppre)
#end
#plt = plot!(X, layout=2)

#anim = @animate for i=2:50
#    plot(range(firstindex(X), lastindex(X), length=i), snratio(X, i), ylims=(0, 50), c=:black, seriestype=:steppre)
#    plot!(N[ref][500:end]./5)
#end
#
#gif(anim, "mygif.gif", fps = 1)
i = 10
plot(range(firstindex(X), lastindex(X), length=i), snratio(X, i), ylims=(0, 50), c=:black, seriestype=:steppre)
plot!(N[ref][500:end]./5)


In [ ]:
plot(X)

In [ ]:
plt = plot([N[1][1000:end], N[2][1000:end]])
savefig(plt, "martin.png")

In [ ]:
using PyCall
signal = pyimport("scipy.signal")
smooth(x) = signal.savgol_filter(x, 31, 4)
N = 100
roi = 750:900
target = M[20]
ref = M[1][roi]
smoothref = smooth(ref)
points = zeros(Float64, (N, N))
shifts = range(-50.0, 50.0, length=N)
gains = range(0.90, 1.10, length=N)
function χ²(target, ref)
    S = @. (ref-target)^2/ref^2
    sum(S[isfinite.(S)])
end
function errfn(x)
    Z = gainshift(target, x...)
    Z = smooth(Z)[roi]
    χ²(Z, smoothref)
end
for i in 1:N, j in 1:N
    points[i, j] = errfn([shifts[i], gains[j]])
end

In [ ]:
#using PyPlot

#fig = figure()
#ax = gca(projection="3d")

#plot3D(repeat(shifts, N), repeat(gains, 10), points)
#Z = plot_surface(shifts, gains, points, alpha=0.9)
#ax[:view_init](30, 180)
#wireframe(shifts, gains, points, alpha=0.5, color=:viridis, stride=1)
contour(shifts, gains, points, levels=100)

In [ ]:
optim = pyimport("scipy.optimize")
guesses = []
converg = []
function cb(x; convergence=0.0)  push!(guesses, x); push!(converg, convergence) end
res = optim.differential_evolution(errfn, ([-50, 50], [0.9, 1.1]), callback=cb)

In [ ]:
Plots.plot(target[roi], seriestype=:steppre, alpha=0.5)
shiftedx = gainshift(target, res["x"]...)
Plots.plot!(ref, seriestype=:steppre, alpha=0.5)
Plots.plot!(shiftedx[roi], seriestype=:steppre, alpha=0.5)
Plots.plot!(smooth(target)[roi], label="target, smooth")
Plots.plot!(smoothref, label="reference, smooth")
Plots.plot!(smooth(shiftedx)[roi], label="target shifted, smooth")

In [ ]:
length(guesses)

In [ ]:
#using PyPlot

#fig = figure()
#ax = gca(projection="3d")

#plot3D(repeat(shifts, N), repeat(gains, 10), points)
#Z = plot_surface(shifts, gains, points, alpha=0.5)
gz = getz(guesses)
gx, gy = collect(zip(guesses...))
#ax.scatter(gx, gy, gz, color=:red)
#ax.view_init(0, 90)
contour(shifts, gains, points', levels=50)
scatter!([x for x in gx], [y for y in gy], markerstrokewidth=0)

In [ ]:
function getz(shiftgain)
    gz = []
    S = collect(shifts)
    G = collect(gains)
    for (shift, gain) in shiftgain
        i = argmin(abs.(shift.-S))
        j = argmin(abs.(gain.-G))
        push!(gz, points[i, j])
    end
    [z for z in gz]
end

In [ ]:
using PyPlot
np = pyimport("numpy")
X, Y = np.meshgrid(shifts, gains)
fig = figure(figsize=(15, 15))
ax = gca(projection="3d")
ax.plot_surface(X, Y, points', alpha=0.6, rstride=2, cmap=:viridis, cstride=1)
ax.view_init(10, 150)
ax.contour(X, Y, points', 50, linewidths=0.5, zdir="z", offset=-4)
ax.scatter(gx, gy, gz, color=:red)

In [ ]:
using Interact

In [ ]:

@manipulate for gain in gains, shift in shifts
    Plots.plot(target[roi], seriestype=:steppre, alpha=0.5)
    Plots.plot!(smoothref, label="reference, smooth")
    Plots.plot!(smooth(target)[roi], label="target, smooth")
    Plots.plot!(ref, seriestype=:steppre, alpha=0.5)
    shiftedx = gainshift(target, shift, gain)
    Plots.plot!(shiftedx[roi], seriestype=:steppre, alpha=0.5)
    p1 = Plots.plot!(smooth(shiftedx)[roi], label="target shifted, smooth", leg=false)
    p2 = Plots.contour(shifts, gains, points', levels=50)
    p2 = Plots.scatter!([shift], [gain])
    l = @layout [a;b]
    plot(p1, p2, layout=l)
end

In [ ]:
res["x"]

# Putting it all together 

In [ ]:
# Load the preprocessed data
events = loadlabr("sirius");
parameters = Parameters("si28.yaml");

# Sort the datafile
ge, gt = makelabr(events, parameters);
mat = copy(ge.matrix);

# Find the good channels
chns = goodchannels(mat);
G = mat[:, chns];
M = [G[:, i] for i in 1:size(G, 2)];
pop!(M);

2000-element Array{Int64,1}:
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  ⋮
  6
 17
 24
 12
 19
 15
 13
 11
 22
 20
 18
 18

In [ ]:
function plotmat(matrix)
    G = convert(Array{Float64, 2}, matrix)
    log10transform!(G)
    pl = heatmap(G', c=:viridis, yticks=([1:length(chns);], chns))
end

function plotstrip(strips)
    G = hcat(strips...)
    G = convert(Array{Float64, 2}, G)
    log10transform!(G)
    pl = heatmap(G', c=:viridis)
end

function log10transform!(G)
    G[G .> 0] .= log10.(G[G .> 0])
    G[G .≤ 0] .= NaN
end

function log10transform(V)
    G = V[1:end]
    G[G .> 0] .= log10.(G[G .> 0])
    G[G .≤ 0] .= NaN
    G
end


log10transform (generic function with 1 method)

In [ ]:
points = localalign(M[13], M[1], [150:1800;])

In [ ]:
plt = plotstrip(M[[1,10]])
@show points
for _points in points
    plt = scatter!(_points, [1,2],markershape=:dtriangle, markercolor=:red, 
        markerstrokewidth=0, legend=false)
end
plt

In [ ]:
refpeaks, targetpeaks = collect(zip(points...)) .|> collect
@show refpeaks
@show shift, gain, gain² = leastsquares(targetpeaks, refpeaks; order=:quadratic)
M′ = gainshift(M[10], shift, gain, gain²)
plotstrip((M[1], M′))

In [ ]:
ref = 1
corrected = [M[ref]]
allpoints = []
shiftgains = []
for i in eachindex(M)
    println("Working on channel $i")
    i == ref && continue
    points = localalign(M[i], M[ref], [80:1900;]; numregions=10)
    push!(allpoints, points)
    refpeaks, targetpeaks = zip(points...) |> collect .|> collect
    shift, gain, gain² = leastsquares(targetpeaks, refpeaks; order=:quadratic)
    push!(shiftgains, [shift, gain, gain²])
    push!(corrected, gainshift(M[i], shift, gain, gain²))
    println("Done working on channel $i")
end

In [ ]:
plt = plotstrip(M)
for (i, _points) in enumerate(allpoints)
    plt = scatter!(_points, [1, i],markershape=:dtriangle, markercolor=:red, 
        markerstrokewidth=0, legend=false)#, size=(1000, 1000))
end
cor = plotstrip(corrected)
display(plt)
display(cor)

In [ ]:
plt = plotstrip(corrected)
signal = pyimport("scipy.signal")
for (i, c) in enumerate(corrected[1:end-1])
    for region in ([60:500;], [500:1000;], [1100:1300;], [1450:1600;])
        peaks = findpeaks(c[1], region, numpeaks=1)
        if i > 1
            corr = signal.correlate(c[1][region], c[region])
            shift = length(region) - argmax(corr)
            peaks1 = peaks .+ shift
        plot!(peaks1, fill(i, length(peaks1)), markershape=:dtriangle, markercolor=:red, 
              legend=false, markerstrokewidth=0)
        else
        plot!(peaks, fill(i, length(peaks)), markershape=:dtriangle, markercolor=:red, 
              legend=false, markerstrokewidth=0)
        end
        
    end
end
plt

In [ ]:
points1 = []
shiftgains = []
for (i, c) in enumerate(corrected)
    println("Working on channel $i")
    i == ref && continue
    points = localalign(c, corrected[ref], [1300:1900;]; numregions=2)
    refpeaks, targetpeaks = zip(points...) |> collect .|> collect
    push!(points1, targetpeaks)
    if length(targetpeaks) < 2
        continue
    end
    push!(shiftgains, leastsquares(targetpeaks, refpeaks; order=:quadratic))
    println("Done working on channel $i")
end


In [ ]:
plt = plotstrip(corrected)
for (i, p) in enumerate(points1)
    plt = plot!(p, fill(i, length(p)), markershape=:dtriangle,
                markercolor=:red, legend=false, markerstrokewidth=0,
                size=(1000, 1000))
end
    
plt

In [ ]:
plt = plotstrip(M)
for (i, (start, stop)) in enumerate(splitregion(80:1900, 10) .|> x -> x .+ 80)
    for j in eachindex(M)
        plt = plot!([start, stop], [j, j], legend=false, color=i)
    end
end
plt

### Trying feature shift

The $\chi^2$ feature minimization works better than cross correlation. I don't know why.
Use it as a starting point for local minimization.

What about an iterative feature match? Once a region is matched, it can be split into subregions for more matching and hence more points.

How are points selected from a region? Peak? How large should a peak-from-region-region be?
Peak might be unstable by same reasons as before. "inverse"-peak selection? Midpoint?

A weakness of these methods is that the shift is always an integer, while in reality
it is a real number.

The feature align works very well _when_ it works. Sometimes there is another minima which
by chance is smaller. Restrict comparison to a window? Punish large shifts?

In [ ]:
function featurematch(reference, feature)
    lref = reference #log10.(ref)
    lfeat = feature #log10.(target)
    window = 1:length(feature)
    errors = zeros(length(lref) - length(feature))
    for shift in 1:(length(lref) - length(feature))
        diff = (feature - lref[window.+shift]).^2
        errors[shift] = sum(diff)
    end
    errors
end
region = 700:900
feature = M[1][region]
shifted = []
for i in eachindex(M)
    # corr = signal.correlate(feature, M[i])
    corr = DSP.xcorr(feature, M[i])
    shift_cor = length(corr)/2 - argmax(corr) |> x -> round(Int64, x)
    #@show length(corr)
    #cor = plot!(corr .|> log10)
    feat = featurematch(M[i], feature) 
    index_feat = argmin(feat)
    #plt = plot!(feat .|> log10)
    shiftplt = plot(region, feature, label="Reference", seriestype=:steppre)
    shiftplt = plot!(region, M[i][region], label="Candidate", seriestype=:steppre)
    shiftplt = plot!(region, M[i][region .- shift_cor], 
                     label="Correlation shift", seriestype=:steppre)
    feat_region = M[i][range(index_feat+1, length=length(region))]
    shiftplt = plot!(region, feat_region,
                     label="χ² shift", seriestype=:steppre)
    push!(shifted, shiftplt)
end
feature = plot(region, feature)
(feature, shifted...) .|> display

In [ ]:
plotpeaks!(peaks, i) = scatter!(peaks, fill(i, length(peaks)), markershape=:dtriangle, markercolor=:red, 
              legend=false, markerstrokewidth=0)

plotpeaks! (generic function with 1 method)

In [ ]:
signal = pyimport("scipy.signal")
smoother(x) = signal.savgol_filter(x, 51, 4)
plt = plotstrip(M)
ref = 1
coeffs = []
corrected = [M[ref]]
allpeaks = []
for i in eachindex(M)
    refpeaks, targetpeaks = featurealign(M[ref], M[i], width=250, searchwidth=50,
                                         roi = 3:1400)
    refpeaks_, targetpeaks_ = featurealign(M[ref], M[i], width=120, searchwidth=500,
                                           roi = 1350:1800, smoother=smoother,
                                           numregions=2)
    push!(refpeaks, refpeaks_...)
    push!(targetpeaks, targetpeaks_...)
    push!(allpeaks, [refpeaks, targetpeaks])
    plt = plotpeaks!(targetpeaks, i)
    i == ref && continue
    shiftgain = leastsquares(targetpeaks, refpeaks; order=2)
    push!(coeffs, shiftgain)
    push!(corrected, gainshift(M[i], shiftgain...))
end
cor = plotstrip(corrected)
display(plt)
display(cor)
#savefig(plt, "/home/erdos/peaks.png")
#savefig(cor, "/home/erdos/quadratic_align.png")
coeffs

In [ ]:
signal = pyimport("scipy.signal")
smoother(x) = signal.savgol_filter(x, 51, 4)
coefficients, alignedspectra = alignspectra(M, lowregion=3:1400, highregion=1350:1800,
                                            highsmoother=smoother)
coefficients, alignedspectra = alignspectra(alignedspectra, lowregion=3:1400, 
                                            highregion=1350:1800, lowsearchwidth=10,
                                            highsearchwidth = 10, lownumregions=15,
                                            highsmoother=smoother)

plotstrip(alignedspectra)

In [ ]:
total = sum(alignedspectra)
plt = plot(xlim=(1500, 2000), ylim=(0, 100))
for spectrum in alignedspectra
    plt = plot!(spectrum)
end
display(plt)
plot(total)

In [ ]:
residuals = []
for ((ref, tar), coeffs) in zip(allpeaks, coeffs)
    f(x) = [x^(i-1)*coeffs[i] for i in eachindex(coeffs)] |> sum
    r̂ = f.(tar)
    push!(residuals, ref - r̂)
end
resplt = plot(size=(1000, 800))
for residual in residuals
    resplt = plot!(residual, marker=:o,  markerstrokewidth=0, legend=false,
                   color=:steelblue, xlabel="Energy [arbitrary unit]",
                   ylabel="Difference (fact - predicted) [arbitrary unit]",
                   title="Residuals of third degree fit")
end
savefig(resplt, "/home/erdos/third_degree_residuals.png")
resplt

In [ ]:
?save

### Trying correlation shift

In [ ]:
roi = 400:600
A, B = M[1][roi], M[3][roi]
plot(roi, A, seriestype=:steppre)
plt = plot!(roi, B, seriestype=:steppre)

signal = pyimport("scipy.signal")
corr = signal.correlate(A, B)
@show shift = length(roi) - argmax(corr)
plt = plot!(roi, gainshift(M[3],-shift, 1.0)[roi], seriestype=:steppre)
cor = plot(corr)
#cor = plot!(argmax(corr), corr[argmax(corr)])
display(cor)
display(plt)

## TODO
- [ ] Correct the alignment
  - Think harder
  - Looks like there is some non-linearity even at high zoom levels.
    Might have to use a linear term
  - If the constant-shift hypothesis holds, try to use cross-correlation instead
  - I __need__ features in order to align. Use all of the highest peaks in the
    reference spectrum and hope it holds?
  - I might have the trade off (negligible higher orders <> weaker features)
  - Use aggressive smoothing in low-data regions?
  - Do a second pass with narrower window around the most prominent features?
  - Either combines polynomials for latter use, or use these points for the
    actual calibration.
  - For high-data regions I can just use peakfinding. Use current
    shift-method for medium-data zones, while for low data zones I need some other
    method. Aggressive gain-shift?
      - Peakfinding only works once the data is already a bit aligned, dumb dumb.
  - Use very small windows after an initial alignment to catch the peaks?
  - The steps seems to be
    - [x] Align the data ok-ish
    - [ ] Catch the peaks/ Make the peaks catchable
    - [ ] Final alignment
- Do I only want alignment, or do I want a _pure_ quadratic alignment? Less work if I
  just do an alignment and assume that it can be arbitrarily approximated by a quadratic by           
  iteration and composition.

In [ ]:
using PyCall
signal = pyimport("scipy.signal")

In [ ]:
p = plot()
for c in corrected[1:end-1]
    p = plot!(signal.savgol_filter(c, 91, 5)[1800:end])
end
p